# 8.3 Reward Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf", split="train")
dataset

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})

In [ ]:
def preprocess(batch):
    result = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(batch["chosen"], batch["rejected"]):
        tokenized_chosen = tokenizer(chosen)
        tokenized_rejected = tokenizer(rejected)

        result["input_ids_chosen"].append(
            tokenized_chosen["input_ids"]
        )
        result["attention_mask_chosen"].append(
            tokenized_chosen["attention_mask"]
        )
        result["input_ids_rejected"].append(
            tokenized_rejected["input_ids"]
        )
        result["attention_mask_rejected"].append(
            tokenized_rejected["attention_mask"]
        )

    return result

dataset = dataset.map(
    preprocess,
    batched=True,
    num_proc=2,
    remove_columns=dataset.column_names
)

max_length = 512
dataset = dataset.filter(
    lambda x: (
        len(x["input_ids_chosen"]) <= max_length
        and len(x["input_ids_rejected"]) <= max_length
    )
)

dataset

In [ ]:
from trl import RewardTrainer, RewardConfig

config = RewardConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,
)

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=config,
    train_dataset=dataset,
)
trainer.train()

# 8.4 SFT: Supervised Fine-Tuning

### 8.3.1 기본 구조

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("imdb", split="train")

sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir="./ckpt",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config,
)
# trainer.train()  # 실제 학습할때만 사용하기

DatacollatorForCompletionOnlyLM

In [4]:
from transformers import AutoTokenizer
from trl import DataCollatorForCompletionOnlyLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
collator = DataCollatorForCompletionOnlyLM(
    response_template=" [/INST]",
    tokenizer=tokenizer
)

prompt_example = "<s>[INST] this is input prompt [/INST] this is response. is it work?</s>"
example = collator([tokenizer(prompt_example)])

label = example.labels[0]
print(label)
print("only response:", tokenizer.decode(label[label > 0]))

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100,   42,   16, 1263,    4,   16,   24,  173,  116,    2])
only response:  this is response. is it work?</s>


In [8]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

prompt = "[INST] this is input prompt [/INST] this is response. is it work?"
print_tokens_with_ids(prompt)

response_template = "[/INST]"
print_tokens_with_ids(response_template)

[('[', 10975), ('INST', 39236), (']', 742), ('Ġthis', 42), ('Ġis', 16), ('Ġinput', 8135), ('Ġprompt', 14302), ('Ġ[/', 48651), ('INST', 39236), (']', 742), ('Ġthis', 42), ('Ġis', 16), ('Ġresponse', 1263), ('.', 4), ('Ġis', 16), ('Ġit', 24), ('Ġwork', 173), ('?', 116)]
[('[/', 48505), ('INST', 39236), (']', 742)]


setup chat format

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
print("before:", tokenizer.chat_template)

model, tokenizer = setup_chat_format(model, tokenizer)
print("after:", tokenizer.chat_template)

before: None
after: {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


formatting func

In [ ]:
from datasets import Dataset


example = [
    {"question": "질문 1", "answer": "답변 1"},
    {"question": "질문 2", "answer": "답변 2"},
    {"question": "질문 3", "answer": "답변 3"},
]
test_dataset = Dataset.from_list(example)

def formatting_prompts_func(sample):
    output_texts = []
    for i in range(len(sample["question"])):
        text = (
            f"### Question: {sample['question'][i]}\n "
            f"### Answer: {sample['answer'][i]}"
        )
        output_texts.append(text)
    return output_texts

sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir="./ckpt",
)
trainer = SFTTrainer(
    model,
    args=sft_config,
    train_dataset=dataset,
)

# trainer.train()  # 실제 학습할때만 사용하기

## packing

In [ ]:
sft_config = SFTConfig(
    packing=True, 
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="./ckpt",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config
)

# trainer.train()  # 실제 학습할때만 사용하기

## model_init_kwargs

In [ ]:
import torch

sft_config = SFTConfig(
    model_init_kwargs={
        "torch_dtype": "bfloat16",
    },
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="./ckpt",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## peft_config

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="./ckpt",
    ),
    peft_config=peft_config
)

# trainer.train()  # 실제 학습할때만 사용하기

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="./ckpt",
    ),
    model_init_kwargs={
        "torch_dtype": torch.bfloat16,
        "load_in_4bit": True,
    },
    peft_config=peft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## model_config

In [7]:
from trl import (
    ModelConfig,
    SFTTrainer,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)

model_config = ModelConfig(
    model_name_or_path="facebook/opt-350m",
    load_in_4bit=True,
    use_peft=True,
)
model_config

ModelConfig(model_name_or_path='facebook/opt-350m', model_revision='main', torch_dtype=None, trust_remote_code=False, attn_implementation=None, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, lora_task_type='CAUSAL_LM', load_in_8bit=False, load_in_4bit=True, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)

In [8]:
quantization_config = get_quantization_config(model_config)
quantization_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float32",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [9]:
get_kbit_device_map()

{'': 0}

In [10]:
peft_config = get_peft_config(model_config)
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [ ]:
torch_dtype = (
    model_config.torch_dtype
    if model_config.torch_dtype in ["auto", None]
    else getattr(torch, model_config.torch_dtype)
)

model_kwargs = dict(
    revision=model_config.model_revision,
    trust_remote_code=model_config.trust_remote_code,
    attn_implementation=model_config.attn_implementation,
    torch_dtype=torch_dtype,
    device_map=(
        get_kbit_device_map() 
        if quantization_config is not None 
        else None
    ),
    quantization_config=quantization_config.to_dict(),
)

training_args = SFTConfig(
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="./ckpt",
    model_init_kwargs=model_kwargs,
)

trainer = SFTTrainer(
    model=model_config.model_name_or_path,
    train_dataset=dataset,
    args=training_args,
    peft_config=peft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## neftune_noise_alpha

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="./ckpt",
        neftune_noise_alpha=5,
    ),
)
# trainer.train()  # 실제 학습할때만 사용하기

# 8.5 PPO: Proximal Policy Optimization


In [1]:
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-5,
    mini_batch_size=1,
    batch_size=1,
)
model = (
    AutoModelForCausalLMWithValueHead
    .from_pretrained(config.model_name)
)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [2]:
from transformers import pipeline

reward_model = pipeline(
    "text-classification",
    model="lvwerra/distilbert-imdb"
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "HuggingFaceH4/cherry_picked_prompts",
    split="train",
)
dataset = dataset.rename_column("prompt", "query")
dataset = dataset.remove_columns(["meta", "completion"])
dataset[0]

{'query': 'Explain the moon landing to a 6 year old in a few sentences.'}

In [4]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"])
    return sample

dataset = dataset.map(tokenize, batched=False)

dataset.set_format(type="torch")

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [5]:
generation_kwargs = {
    "max_length": 256,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

dataset = dataset.filter(
    lambda x: len(x["input_ids"]) <= generation_kwargs['max_length']
)

In [6]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
    data_collator=collator,
)

In [ ]:
import torch
from tqdm.notebook import tqdm


epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader):
        query_tensors = batch["input_ids"]

        # Rollout: 학습할 모델로 문장 생성
        response_tensors = ppo_trainer.generate(
            query_tensors,
            **generation_kwargs
        )
        batch["response"] = [
            tokenizer.decode(r.squeeze())
            for r in response_tensors
        ]

        # Evaluate: Reward 모델로 점수 부여
        # return_full_text 옵션 입력이 불가능하므로, response == query + gen_text
        pipe_outputs = reward_model(batch["response"])
        rewards = [
            torch.tensor(output["score"])
            for output in pipe_outputs
        ]

        # Optimization: ppo 학습 진행
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

# 모델 저장
ppo_trainer.save_pretrained("/content/drive/MyDrive/Books/outputs/my_ppo_model")

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1300: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std_scores = data["scores"].std()
/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1327: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/home/nsw/.pyenv/versions/3.10.

# 8.6 Best of N Sampling

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, GenerationConfig
from trl import AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from trl.extras import BestOfNSampler

ref_model_name = "gpt2"
reward_model_name = "gpt2"
device = torch.device("cuda")
ref_model = (
    AutoModelForCausalLMWithValueHead
    .from_pretrained(ref_model_name)
    .to(device)
)
tokenizer = AutoTokenizer.from_pretrained(ref_model_name)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model_name,
    device=device
)

def queries_to_scores(list_of_strings):
    return [output["score"] for output in reward_pipe(list_of_strings)]

best_of_n = BestOfNSampler(
    ref_model,
    tokenizer,
    queries_to_scores,
    length_sampler=LengthSampler(10, 128),
    sample_size=5,
    n_candidates=2,
    generation_config=GenerationConfig(
        min_length= -1,
        top_k=0.0,
        top_p= 1.0,
        do_sample= True,
        pad_token_id=tokenizer.eos_token_id
    ),
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
result = best_of_n.generate(
    tokenizer("what is love?", return_tensors="pt").input_ids[0],
    device=device
)

for r in result[0]:
    print(r)
    print("=" * 50, "\n")

what is love? Love? Where can I find it?

Love love, what are you love loving?

Love love, what is the Holiness to you?

Love feeling love, what is the Glory of God in you?

Love love, what does it mean to love?

Love. Love who has

what is love? How has love been changed? How things would change under God? Do God's desires continue with the child? Does love diminish with society's powers? Should people control themselves? At times these questions have seemed theological, yet of all the demanding questions that Jesus confronts, surely we get the most devastated reading.

John Piper's



# 8.7 DPO: Directi Preference Optimization


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
ref_model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name)

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train",
)
dataset

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 160800
})

In [3]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/160800 [00:00<?, ? examples/s]

In [ ]:
from trl import DPOConfig, DPOTrainer

args = DPOConfig(
    beta=0.1,
    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,
    output_dir="./ckpt",
)

trainer = DPOTrainer(
    model, 
    ref_model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=args,
)

trainer.train()

# 8.8 KTO: Kahneman-Tversky Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "trl-lib/qwen1.5-1.8b-sft"
ref_model_name = "trl-lib/qwen1.5-1.8b-sft"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("trl-lib/kto-mix-14k", split="train")
dataset

Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 13500
})

In [ ]:
def process(row):
    row["prompt"] = tokenizer.apply_chat_template(
        row["prompt"], tokenize=False
    )
    row["completion"] = tokenizer.apply_chat_template(
        row["completion"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)
print(dataset[1]['completion'])

Map (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

<|im_start|>assistant
No, the answer provided does not directly answer the question regarding the age of Julio Cesar Chavez when he fought Oscar De La Hoya. The provided information describes some general records held by Julio Cesar Chavez throughout his career. To answer your original question, let me provide the relevant information:

Julio Cesar Chavez fought Oscar De La Hoya on June 7, 1996, in a match called "Ultimate Glory." Chavez was born on July 12, 1962. To calculate his age at the time of the fight, we need to find the difference between the fight date and his birthdate.

From July 12, 1962, to June 7, 1996, there are:
- 33 years (from 1962 to 1995)
- An additional year from his birthday in 1995 (July 12, 1995) to the fight date in 1996 (June 7, 1996), which is approximately 10 months and 26 days.

Therefore, Julio Cesar Chavez was about 33 years and 10 months old when he fought Oscar De La Hoya.<|im_end|>



/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from trl import KTOTrainer, KTOConfig

args = KTOConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

    max_length=512,
    max_prompt_length=512,
    remove_unused_columns=False,
    dataset_num_proc=2,

    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
)

trainer = KTOTrainer(
    model,
    ref_model,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

# 8.9 CPO: Contrastive Preference Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 160800
})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/160800 [00:00<?, ? examples/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from trl import CPOConfig, CPOTrainer

args = CPOConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,

    beta=0.1,
)

trainer = CPOTrainer(
    model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=dataset,
)

trainer.train()

# 8.10 ORPO: Odds Ratio Preference Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 160800
})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/160800 [00:00<?, ? examples/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from trl import ORPOConfig, ORPOTrainer

args = ORPOConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,

    beta=0.1,
)

trainer = ORPOTrainer(
    model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=dataset,
)

trainer.train()